In [13]:
import rasterio as rio
import rioxarray as rxr
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import MultiPolygon
thomas_fire_bs = r'D:/imagery/masters/output/S2_tf_dNBR.tif'
thomas_fire_shp = r'D:/imagery/masters/baer/thm2017_Basin_DFPredictions_15min_12mmh.shp'


In [14]:
shape = gpd.read_file(thomas_fire_shp)
print(shape)
fire = rxr.open_rasterio(thomas_fire_bs)

      BASIN_ID  Shape_Leng    Shape_Area  Segment_ID  Acc_Cl  Perim_Buff  \
0          123     16060.0  6.322700e+06         123     1.0          10   
1          155      3140.0  2.716000e+05         155     1.0          10   
2          167      1800.0  7.260000e+04         167     1.0          10   
3          172      2720.0  2.340000e+05         172     1.0          10   
4          184      1540.0  4.900000e+04         184     1.0          10   
...        ...         ...           ...         ...     ...         ...   
1731     44218      2880.0  2.577000e+05       44218     1.0          10   
1732     44238      6260.0  8.933000e+05       44238     1.0          10   
1733     44256      1140.0  3.360000e+04       44256     1.0          10   
1734     44320      3020.0  1.722000e+05       44320     1.0          10   
1735     44339      7960.0  1.424200e+06       44339     1.0          10   

      InsidePeri  OutsidePer  Fire_ID     Fire_Segme  ...       LnV  \
0              0

In [17]:
for idx, row in shape.iterrows():
    basin_id = row['BASIN_ID']
    geometry = row['geometry']
    basin_dNBR = fire.rio.clip(geometry, shape.crs, all_touched=False, drop=True, invert=False)

d:\Code\PHA_burn_severity\.conda\Lib\site-packages\rasterio\features.py:328: ShapeSkipWarning: Invalid or empty shape type at index 0 will not be rasterized.
  warnings.warn('Invalid or empty shape {} at index {} will not be rasterized.'.format(geom, index), ShapeSkipWarning)
d:\Code\PHA_burn_severity\.conda\Lib\site-packages\rasterio\features.py:328: ShapeSkipWarning: Invalid or empty shape coordinates at index 1 will not be rasterized.
  warnings.warn('Invalid or empty shape {} at index {} will not be rasterized.'.format(geom, index), ShapeSkipWarning)


ValueError: No valid geometry objects found for rasterize